In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
col_names =['buying price', 'maintenance cost', 'number of doors', 
            'number of persons', 'lug_boot' ,'safety', 'decision']

df = pd.read_csv('../data/car_evaluation.csv', header=None, names=col_names)                
df  

,buying price,maintenance cost,number of doors,number of persons,lug_boot,safety,decision
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [4]:
X = df.drop(['decision'], axis=1)

y = df['decision']

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [6]:
import category_encoders as ce

encoder = ce.one_hot.OneHotEncoder(cols=['buying price', 'maintenance cost', 'number of doors', 
            'number of persons', 'lug_boot' ,'safety',])


X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

### Задача
Обучить random forest и посмотреть на матрицу ошибок

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


rfc = RandomForestClassifier(random_state=0, n_estimators=500)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[120,   7,   0,   2],
       [  2,  17,   0,   1],
       [  4,   0, 393,   0],
       [  3,   1,   0,  21]])

In [9]:
print("Метрики качества на отложенной выборке")

print("Accuracy: {0:.4f}".format(
    accuracy_score(y_test, y_pred)
))

print("Precision: {0:.4f}".format(
    precision_score(y_test, y_pred, average='macro')
))

print("Recall {0:.4f}".format(
    recall_score(y_test, y_pred, average='macro')
))

print("F1 score: {0:.4f}".format(
    f1_score(y_test, y_pred, average='macro')
))

Метрики качества на отложенной выборке
Accuracy: 0.9650
Precision: 0.8713
Recall 0.9025
F1 score: 0.8845


## Задача

Обучить модель градиентного бустинга на датасете, провести подбор гиперпараметров (взять любые 3 штуки, их список можно получить через документацию к модели).

Сравнить качество с полученными ранее результатами.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [11]:
model = GradientBoostingClassifier()
params = {
    'loss': ['deviance', 'exponential'],
    'n_estimators': [100, 200],
    'learning_rate': [0.05]
}

cls = GridSearchCV(model, params, cv=5)
cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)

/Users/sabzero43/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/sabzero43/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sabzero43/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 448, in fit
    self._check_params()
  File "/Users/sabzero43/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 249, in _check_params
    self.loss_ = loss_class(self.n_classes_)
  File "/Users/sabzero43/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb_losses.py", line 775, in __init__
    raise ValueError("{0:s} requires 2 classes; got {1:d} class(es)"
ValueError: ExponentialLoss requires 2 classes; got 4 c

In [13]:
confusion_matrix(y_test, y_pred)

array([[118,   8,   2,   1],
       [  0,  19,   0,   1],
       [  1,   0, 396,   0],
       [  1,   0,   0,  24]])

In [12]:
print("Метрики качества на отложенной выборке")

print("Accuracy: {0:.4f}".format(
    accuracy_score(y_test, y_pred)
))

print("Precision: {0:.4f}".format(
    precision_score(y_test, y_pred, average='macro')
))

print("Recall {0:.4f}".format(
    recall_score(y_test, y_pred, average='macro')
))

print("F1 score: {0:.4f}".format(
    f1_score(y_test, y_pred, average='macro')
))

Метрики качества на отложенной выборке
Accuracy: 0.9755
Precision: 0.9013
Recall 0.9556
F1 score: 0.9234


### Задача
Добавить в стекинг еще несколько классификаторов (можно взять классификаторы из прошлых заданий) и получить финальный результат.

In [14]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

estimators = [
    ('LogReg', LogisticRegression()),
    ('Tree', DecisionTreeClassifier()),
    ('RF', RandomForestClassifier())
]
clf=StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[116,   5,   5,   3],
       [  0,  19,   0,   1],
       [  3,   0, 394,   0],
       [  3,   2,   0,  20]])

In [15]:
print("Метрики качества на отложенной выборке")

print("Accuracy: {0:.4f}".format(
    accuracy_score(y_test, y_pred)
))

print("Precision: {0:.4f}".format(
    precision_score(y_test, y_pred, average='macro')
))

print("Recall {0:.4f}".format(
    recall_score(y_test, y_pred, average='macro')
))

print("F1 score: {0:.4f}".format(
    f1_score(y_test, y_pred, average='macro')
))

Метрики качества на отложенной выборке
Accuracy: 0.9615
Precision: 0.8756
Recall 0.9104
F1 score: 0.8892
